In [26]:
import requests
import json
from config import api_key
from pprint import pprint
import time
from random import randint

In [24]:
api_key

'BQAYs3t3RdDcG6b2C5BWVHnW6G961q_zBe57EV9GjiA6E56R2EZAyIkimZhHJ3ZgSo6RQxRqZOGiaLiIvxfZj-NalYhDvfsgBVDpqz0LMw3geqJb4sh7-36JL-79ZC_29EcW2mP6UzOAAJiCSAuI'

SPOTIFY DOCUMENTATION
https://developer.spotify.com/console/tracks/

Example: wk06, 02, 07

In [25]:
#[create list of songs from database to query]
import pandas as pd
import sqlite3

songs_csv = "../../Resources/spotify_tracks_features.csv"
df = pd.read_csv(songs_csv)
df.columns = df.columns.str.strip()
df.columns = df.columns.str.replace(".","")

song_ids = df['id'].tolist()

song_ids

<ipython-input-25-a62b3f009a48>:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace(".","")


['7lmeHLHBe4nmXzuXc0HDjk',
 '1wsRitfRRtWyEapl0q22o8',
 '1hR0fIFK2qRG3f3RF70pb7',
 '2lbASgTSoDO7MTuLAXlTW0',
 '1MQTmpYOZ6fcMQc56Hdo7T',
 '2LXPNLSMAauNJfnC58lSqY',
 '3moeHk8eIajvUEzVocXukf',
 '4llunZfVXv3NvUzXVB3VVL',
 '21Mq0NzFoVRvOmLTOnJjng',
 '6s2FgJbnnMwFTpWJZzvb6z',
 '7o2Razxnl33Jrgz4PpMYNa',
 '3Hxu3Ey1TV9mZ5XqMXckiS',
 '6ZU9RJIZ0fNaFuQM57bDIA',
 '3FUS56gKr9mVBmzvlnodlh',
 '3tTL7jlSkowXidYeafFtwG',
 '2vuDdXqekkDCSdawJyUpT6',
 '11cxKUEgnVAlesUKt4e3br',
 '1IDAJagxB9AQjjYXaiDK1j',
 '6zbHSDJjgrNdfIxPyGfPBt',
 '3YEk8mVdMI7rxtfimlUd1G',
 '0WK0EqiidP6WEDOHK34HEe',
 '1zVE9JBBy8j0KmlbM8Xwhi',
 '2SwgVZn9S4NGueAaEAryf1',
 '0QCQ1Isa0YPVyIbs6JwpO1',
 '3kIBEFhsZOeeKGebxRraOb',
 '5dNDRw6qjDcnbW3luRhElU',
 '561UU4MvlsCenN1x7leYCh',
 '5ytl6Ay8G7ucmujk9R3S0c',
 '0zjqm0VWOQT9is5gT9CTib',
 '5UMTzSiB8mMu9vuo2PZkhd',
 '6Uz62Rd9XadpjT8Zebgwq2',
 '7IYXfiXsiMdhcboT6Vw7Qu',
 '0dT5eHAqMNchKMw06dgyFW',
 '3nvfZxZcdZ5rIy8bxAr5fX',
 '2nWM35FPxkYAWHHv9hrczL',
 '1pxJE4N74aGkC6BbIUhq3P',
 '5oLsi3F0C26MGnXJcnzQyB',
 

In [28]:
import requests
from requests.structures import CaseInsensitiveDict
import numpy as np

url = "https://api.spotify.com/v1/tracks/3JbxyXrJDf4tNLvIQI2iXl"
headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"
headers["Authorization"] = f"Bearer {api_key}"

response = requests.get(url, headers=headers).json()

response

{'error': {'status': 401, 'message': 'The access token expired'}}

In [27]:
import requests
from requests.structures import CaseInsensitiveDict
import numpy as np

# set up lists to hold reponse info
popularity = []


url = "https://api.spotify.com/v1/tracks/"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"
headers["Authorization"] = f"Bearer {api_key}"

# Loop through the list of songs and perform a request for data on each
for song in song_ids:
    # resp = requests.get(query_url, headers=headers)
    try:
        response = requests.get(url + song, headers=headers).json()
        popularity.append(response['popularity'])
        print(f"Song popularity for {song}: {response['popularity']}")
    except:
        time.sleep(31)
        try:
            response = requests.get(url + song, headers=headers).json()
            popularity.append(response['popularity'])
            print(f"Song popularity for {song}: {response['popularity']}")
        except:
            popularity.append(np.nan)
            print(response.status_code)

        # print(f"Song popularity for {popularity}:")

    time.sleep(randint(1,5))

# print(resp.status_code)
# resp.json()
# print(response['popularity'])
# print(api_key)

Song popularity for 7lmeHLHBe4nmXzuXc0HDjk: 65
Song popularity for 1wsRitfRRtWyEapl0q22o8: 69
Song popularity for 1hR0fIFK2qRG3f3RF70pb7: 55
Song popularity for 2lbASgTSoDO7MTuLAXlTW0: 50
Song popularity for 1MQTmpYOZ6fcMQc56Hdo7T: 66
Song popularity for 2LXPNLSMAauNJfnC58lSqY: 53
Song popularity for 3moeHk8eIajvUEzVocXukf: 43
Song popularity for 4llunZfVXv3NvUzXVB3VVL: 46
Song popularity for 21Mq0NzFoVRvOmLTOnJjng: 43
Song popularity for 6s2FgJbnnMwFTpWJZzvb6z: 43
Song popularity for 7o2Razxnl33Jrgz4PpMYNa: 43
Song popularity for 3Hxu3Ey1TV9mZ5XqMXckiS: 43
Song popularity for 6ZU9RJIZ0fNaFuQM57bDIA: 51
Song popularity for 3FUS56gKr9mVBmzvlnodlh: 57
Song popularity for 3tTL7jlSkowXidYeafFtwG: 48
Song popularity for 2vuDdXqekkDCSdawJyUpT6: 43
Song popularity for 11cxKUEgnVAlesUKt4e3br: 47
Song popularity for 1IDAJagxB9AQjjYXaiDK1j: 49
Song popularity for 6zbHSDJjgrNdfIxPyGfPBt: 47
Song popularity for 3YEk8mVdMI7rxtfimlUd1G: 44
Song popularity for 0WK0EqiidP6WEDOHK34HEe: 42
Song populari

AttributeError: 'dict' object has no attribute 'status_code'

In [6]:
df_popularity = pd.DataFrame(popularity, columns=['popularity'])
df_popularity.to_csv('../../Resources/popularity.csv')